In [1]:
import pysam
import pandas as pd
from tqdm.auto import tqdm
import numpy as np
import itertools

In [6]:
bam_file = "/home/dbeb/btech/bb1160039/scratch/project/Datasets/Human PBMC/pbmc_10k_protein_v3_possorted_genome_bam.bam"
bai_file = "/home/dbeb/btech/bb1160039/scratch/project/Datasets/Human PBMC/pbmc_10k_protein_v3_possorted_genome_bam.bam.bai"

In [7]:
samf = pysam.Samfile(bam_file, "rb")
replicon_dict = dict([[replicon, {'seq_start_pos': 0,'seq_end_pos': length}] for replicon, length in zip(samf.references, samf.lengths)])

In [8]:
print(replicon_dict['1']['seq_start_pos'])
print(replicon_dict['1']['seq_end_pos'])

0
248956422


In [9]:
samfile = pysam.AlignmentFile(bam_file, "rb", index_filename = bai_file)

In [10]:
#read all non-genes
x=['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21','22','X','Y']
list_tags_all = []

for i in tqdm(range(0,len(x))):
    for read in samfile.fetch(x[i]):
        try:
            if (read.has_tag("GX")==False and read.get_tag("NH")==1):
                list_tags_all.append([read.get_tag("CB"),read.get_tag("UB"),"gene" + "-" + str(read.reference_name) + "-" +str(read.get_reference_positions()[0])+'-'+ str(read.get_reference_positions()[-1])])
        except KeyError:
            continue

In [11]:
%%time
list_tags_all_rm_dup = list(list_tags_all for list_tags_all,_ in itertools.groupby(list_tags_all))

CPU times: user 16.9 s, sys: 5.15 s, total: 22.1 s
Wall time: 22.2 s


In [12]:
print(len(list_tags_all))
print(len(list_tags_all_rm_dup))

90108371
72662073


In [13]:
start=0
val=int(list_tags_all_rm_dup[start][2].split("-")[2])
collapse_len = 300

for i in tqdm(range(1,len(list_tags_all_rm_dup))):
    if (int(list_tags_rm_dup[i][2].split("-")[2]) - val>collapse_len):
        if (i-start>1):
            #update the inner elements
            s=list_tags_rm_dup[start][2].split("-")
            t=list_tags_rm_dup[i-1][2].split("-")
            new=s[0]+"-"+s[1]+"-"+s[2]+"-"+str(int(t[3]))
            for inner in list_tags_all_rm_dup[start:i]:
                inner[2] = new
        #update start to point to this pos
        start = i
        #update val to the val at this pos
        val = int(list_tags_all_rm_dup[i][2].split("-")[2])

In [14]:
%%time
list_tags_all_rm_dup_final = list(list_tags_all_rm_dup for list_tags_all_rm_dup,_ in itertools.groupby(list_tags_all_rm_dup))

CPU times: user 14.3 s, sys: 3.22 s, total: 17.6 s
Wall time: 17.6 s


In [16]:
n=len(list_tags_all_rm_dup_final)
n

72249188

In [12]:
%%time
bins = 20
df_all = []
for i in range(bins):
    df_all.append(pd.DataFrame(list_tags_rm_dup_final[:int(n/bins)], columns=["celltag","moltag","pseudoname"]))

CPU times: user 48.8 s, sys: 48.9 s, total: 1min 37s
Wall time: 9min 55s


In [13]:
%%time
celltags = []
pseudonames = []
for i in range(bins):
    celltags.append(df_all[i]['celltag'].value_counts())
    pseudonames.append(df_all[i]['pseudoname'].value_counts())

CPU times: user 1min 48s, sys: 10.1 s, total: 1min 58s
Wall time: 2min 1s


In [14]:
%time
df_all_subset = []
cutoff = 0
for i in range(bins):
    temp = df_all[i][(df_all[i]["celltag"].isin(celltags[i][celltags[i]>cutoff].index)) & 
                     (df_all[i]["pseudoname"].isin(pseudonames[i][pseudonames[i]>0].index))]
    df_all_subset.append(temp)

CPU times: user 9 µs, sys: 1e+03 ns, total: 10 µs
Wall time: 13.8 µs


In [15]:
%%time
counts_list = []
for i in range(bins):
    counts_list.append(df_all_subset[i].groupby(['pseudoname','celltag']).size().unstack('celltag', fill_value=0))

CPU times: user 27.1 s, sys: 2min 22s, total: 2min 49s
Wall time: 7min 47s


In [16]:
counts_list_columns = [counts_list[i].columns for i in range(bins)]
cell_bcode= sorted(set(counts_list_columns[0]).intersection(*counts_list_columns))

In [17]:
len(cell_bcode)

7996

In [18]:
%%time
for i in range(bins):
    counts_list[i] = counts_list[i][cell_bcode]

CPU times: user 16.2 s, sys: 1min 2s, total: 1min 18s
Wall time: 2min 28s


In [ ]:
%%time
counts_df = pd.concat(counts_list)

In [ ]:
counts_df_final=counts_df.groupby(level=0, axis=0).sum()